In [ ]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-ma

# BERT fine tuning con testa di classificazione per Sequence Classification

Questo file è stato utilizzato solo su Colab per poter sfruttare le GPU, tutti i path quindi si riferiscono a quella posizione, e non funzionano su questa cartella. Non sono stati cambiati e non è stato rieseguito in locale per non cambiare lo stato del notebook dell'addestramento del modello usato per i test.

In [ ]:
import os
import numpy as np
import pandas as pd
import datasets
import evaluate
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
dir_path_train = '/content/drive/MyDrive/TED/train/'
dir_path_test = '/content/drive/MyDrive/TED/test/'

# definisco le liste dei nomi dei file del training e del test
file_names_train = [f'{dir_path_train}{n}_0.txt' for n in range(0,4000)] + [f'{dir_path_train}{n}_1.txt' for n in range(4000,8000)]
labels_train = [0]*4000 + [1] *4000

file_names_test = [f'{dir_path_test}{n}_0.txt' for n in range(0,400)] + [f'{dir_path_test}{n}_1.txt' for n in range(400,800)]
labels_test = [0]*400 + [1] *400

In [ ]:
def extract_texts(file_names, labels):
  # funzione che per ogni file crea un dizionario con id, testo e label, restituendoli poi in una lista 
  data = []
  for i in range(len(file_names)):
    f_name = file_names[i]
    with open(f_name, 'r', encoding='utf-8') as f:
      text = f.read()
    id = str(i)
    label = labels[i]

    data.append({
        'id': id,
        'sentence': text,
        'label': label
    })
    if i%100 == 0:
      print(i)

  return data


In [ ]:
train_data = extract_texts(file_names_train, labels_train)
test_data = extract_texts(file_names_test, labels_test)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
0
100
200
300
400
500
600
700


In [ ]:
# Trasformo i dataset nel formato di huggingface
train = datasets.Dataset.from_pandas(pd.DataFrame(data=train_data))
test = datasets.Dataset.from_pandas(pd.DataFrame(data=test_data))

# ricavo il validation set
train_val = train.train_test_split(test_size=0.1, seed=8)
train = train_val["train"]
val = train_val["test"]

In [ ]:
print(train, val, test)

Dataset({
    features: ['id', 'sentence', 'label'],
    num_rows: 7200
}) Dataset({
    features: ['id', 'sentence', 'label'],
    num_rows: 800
}) Dataset({
    features: ['id', 'sentence', 'label'],
    num_rows: 800
})


In [ ]:
# nome modello
model_name = 'dbmdz/bert-base-italian-cased'

# carico il modello
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# tokenizzatore associato
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
    tokens = tokenizer(batch['sentence'], padding=True, truncation=True, max_length=512)
    tokens['label'] = [int(label) for label in batch["label"]]

    return tokens

# tokenizzo i dataset
train = train.map(tokenize, batched=True)
val = val.map(tokenize, batched=True)
test = test.map(tokenize, batched=True)

train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
# salvo i dataset
train.save_to_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_train')
val.save_to_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_val')
test.save_to_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_test')

Saving the dataset (0/1 shards):   0%|          | 0/7200 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/800 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
# uso questa cella per non dover rifare ogni esecuzione tutta la parte di estrazione dei testi e tokenizzazione
train = datasets.load_from_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_train')
val = datasets.load_from_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_val')
test = datasets.load_from_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_test')

In [ ]:
test

Dataset({
    features: ['id', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 800
})

In [ ]:

num_epochs = 5

training_args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    logging_strategy="epoch",
    save_strategy = "epoch",
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [ ]:
def acc_metric(eval_pred):
  accuracy_metric = evaluate.load("accuracy")
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)

  return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:

# fine-tuning del modello
trainer = Trainer(
    model,
    training_args,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=tokenizer,
    compute_metrics=acc_metric
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.692900,0.612420,0.676250
2,0.597400,0.594840,0.687500
3,0.453600,0.671662,0.663750
4,0.331500,0.782270,0.685000
5,0.250300,0.866667,0.677500


TrainOutput(global_step=2250, training_loss=0.4651334635416667, metrics={'train_runtime': 1528.478, 'train_samples_per_second': 23.553, 'train_steps_per_second': 1.472, 'total_flos': 7887732328653120.0, 'train_loss': 0.4651334635416667, 'epoch': 5.0})

In [ ]:
# salvo il modello
trainer.save_model("/content/drive/My Drive/FINETUNED_MODEL")

In [ ]:
log_history = trainer.state.log_history

In [ ]:
# salvo i log del modello
import json
with open('/content/drive/My Drive/finetuned_model_logs.json', 'w') as f:
    json.dump(log_history, f)